### Example of TSSOS failure to find solution on 4-th degree polynomial

Consider simplest possible Lindblad master equation with just one dissipator:

$
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]
$,

where Hamiltonian is 


$
    H_0 = |1\rangle \langle 1| = \begin{pmatrix} h & 0 \\ 0 & 0
   \end{pmatrix}
$

$
A = \sqrt{\gamma} \sigma = \begin{pmatrix} 0 & \sqrt{\gamma} \\ 0 & 0
   \end{pmatrix} = \begin{pmatrix} 0 & a \\ 0 & 0
   \end{pmatrix}$

$\sigma = |0\rangle \langle 1|  = \begin{pmatrix} 0 & 1 \\ 0 & 0
   \end{pmatrix}$


In [1]:
using DynamicPolynomials

@polyvar E

    Hˢʸᵐᵇ = [ E   0
              0   0. ]

@polyvar a

    Aˢʸᵐᵇ = [ 0  a
              0  0.]

2×2 Matrix{Term{true, Float64}}:
 0.0  a
 0.0  0.0

Load exact data and essemble objective

In [2]:
include("LiPoSID.jl")

┌ Info: Precompiling TSSOS [81648402-ffe6-11e9-2394-0de17a9afdad]
└ @ Base loading.jl:1423
┌ Warning: Package TSSOS does not have Printf in its dependencies:
│ - If you have TSSOS checked out for development and have
│   added Printf as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with TSSOS
└ Loading Printf into TSSOS from project dependency, future warnings for TSSOS are suppressed.


Main.LiPoSID

In [3]:
parentdir = pwd()

γᵗˣᵗ = "2.5133"

datadir = parentdir*"\\DATA\\"

ρᵍ, tᵍ = LiPoSID.get_rho_series(datadir*"State_B1_2CUT_data.h5", γᵗˣᵗ)
ρᵉ, tᵉ = LiPoSID.get_rho_series(datadir*"State_B2_2CUT_data.h5", γᵗˣᵗ)
ρˣ, tˣ = LiPoSID.get_rho_series(datadir*"State_B3_2CUT_data.h5", γᵗˣᵗ)
ρʸ, tʸ = LiPoSID.get_rho_series(datadir*"State_B4_2CUT_data.h5", γᵗˣᵗ)

ρˣ = convert(Vector{Matrix{ComplexF64}}, ρˣ)
ρʸ = convert(Vector{Matrix{ComplexF64}}, ρʸ)
ρᵍ = convert(Vector{Matrix{ComplexF64}}, ρᵍ)
ρᵉ = convert(Vector{Matrix{ComplexF64}}, ρᵉ)
t = convert(Vector{Float64}, tˣ);

obj = LiPoSID.simpson_obj(ρᵉ, t, Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
obj += LiPoSID.simpson_obj(ρᵍ, t, Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
obj += LiPoSID.simpson_obj(ρˣ, t, Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
obj += LiPoSID.simpson_obj(ρʸ, t, Hˢʸᵐᵇ, [Aˢʸᵐᵇ])

0.0031063757386775343a⁴ + 0.001731603372194455E² - 0.015158874971351882a² - 0.08730024647750814E + 1.121995426822635

Here we have polynomial of degree 4 which is symmetric in valiable $a$. 

It looks like it is somehow patalogic for TSSOS :(

In [4]:
using TSSOS

In [5]:
opt,sol,data = tssos_first(obj, variables(obj), QUIET=true, solution=true);
previous_sol = sol

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.0031729910612705863

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Global optimality certified with relative optimality gap 0.000002%!


2-element Vector{Float64}:
 25.207922287328664
  1.5620376482685465

In [6]:
#opt,sol,data = tssos_first(obj, variables(obj), QUIET=true, solution=true, newton=false);
#previous_sol = sol

In [7]:
#opt, sol, data = tssos_higher!(data; QUIET=true, solution=true);

In [8]:
sol

2-element Vector{Float64}:
 25.207922287328664
  1.5620376482685465

In [9]:
solution = variables(obj) => sol

PolyVar{true}[E, a] => [25.207922287328664, 1.5620376482685465]

In [10]:
subs(Hˢʸᵐᵇ, solution)

2×2 Matrix{Term{true, Float64}}:
 25.2079  0.0
 0.0      0.0

In [11]:
subs(Aˢʸᵐᵇ, solution)

2×2 Matrix{Term{true, Float64}}:
 0.0  1.56204
 0.0  0.0

The dissipator is almoust zero which is simply not true!!!

### Reformulate the same problem 

Lindblad master equation could be rewritten in even simpler form if put $\gamma$ out of brackets:

$
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]
$

$
A = \sqrt{\gamma} \sigma = \begin{pmatrix} 0 & \sqrt{\gamma} \\ 0 & 0
   \end{pmatrix} = \begin{pmatrix} 0 & a \\ 0 & 0
   \end{pmatrix}$
   
$
    \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+ \gamma \left[\sigma \rho \sigma^\dagger - \frac{1}{2} \left\{ \sigma^\dagger \sigma, \rho \right\} \right]
$


$\sigma = |0\rangle \langle 1|  = \begin{pmatrix} 0 & 1 \\ 0 & 0
   \end{pmatrix}$

In [12]:
function γ_rhs(ρ, H, γ)
    """
    Right hand side of the Lindblad master equation
    """
    
    𝜎 = [ 0  1
          0  0. + 0im ]
    
    return -im * (H * ρ - ρ * H) + γ * (𝜎 * ρ * 𝜎' - (𝜎' * 𝜎  * ρ + ρ * 𝜎' * 𝜎) / 2)
    
end

function γ_simpsom_obj(ρ::Vector{Matrix{ComplexF64}}, t, H, γ)
    
    obj = 0
    for i in 3:length(ρ)
        obj += LiPoSID.frobenius_norm2(
            ρ[i] - ρ[i-2] - (t[i]-t[i-1])γ_rhs((ρ[i-2] + 4ρ[i-1] + ρ[i])/3, H, γ)
        )
    end
    obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    return obj
end

γ_simpsom_obj (generic function with 1 method)

In [13]:
γᶠˡᵒᵃᵗ = parse(Float64, γᵗˣᵗ)

2.5133

In [14]:
𝜎 = [ 0  1
      0  0. + 0im ]

obj_γ = γ_simpsom_obj(ρᵉ, t, Hˢʸᵐᵇ, γᶠˡᵒᵃᵗ)
obj_γ += γ_simpsom_obj(ρᵍ, t, Hˢʸᵐᵇ, γᶠˡᵒᵃᵗ)
obj_γ += γ_simpsom_obj(ρˣ, t, Hˢʸᵐᵇ, γᶠˡᵒᵃᵗ)
obj_γ += γ_simpsom_obj(ρʸ, t, Hˢʸᵐᵇ, γᶠˡᵒᵃᵗ)

0.001731603372194455E² - 0.08730024647750813E + 1.1035185981973

which is of course very simple quadratic polynomial

In [15]:
opt_γ, sol_γ, data_γ = tssos_first(obj_γ, variables(obj_γ), QUIET=true, solution=true, newton=false);|

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.00318969210985631
Global optimality certified with relative optimality gap 0.000001%!


| (generic function with 36 methods)

The Hamiltonian remains almoust the same:

In [16]:
solution = variables(obj_γ) => sol_γ
subs(Hˢʸᵐᵇ, solution)

2×2 Matrix{Term{true, Float64}}:
 25.2081  0.0
 0.0      0.0

but we can find another dissipator $A$ that can fit the data much better

In [17]:
√γᶠˡᵒᵃᵗ*𝜎

2×2 Matrix{ComplexF64}:
 0.0+0.0im  1.58534+0.0im
 0.0+0.0im      0.0+0.0im

Thus for some reason TSSOS fails to find the right solution :(

#### We can also solve it as 2D optimization problem in $E$ and $\gamma$

In [18]:
@polyvar γ

(γ,)

In [19]:
obj_γˢʸᵐᵇ = γ_simpsom_obj(ρᵉ, t, Hˢʸᵐᵇ, γ)
obj_γˢʸᵐᵇ += γ_simpsom_obj(ρᵍ, t, Hˢʸᵐᵇ, γ)
obj_γˢʸᵐᵇ += γ_simpsom_obj(ρˣ, t, Hˢʸᵐᵇ, γ)
obj_γˢʸᵐᵇ += γ_simpsom_obj(ρʸ, t, Hˢʸᵐᵇ, γ)

0.001731603372194455E² + 0.0031063757386775343γ² - 0.08730024647750814E - 0.015158874971351882γ + 1.121995426822635

In [20]:
opt_γˢʸᵐᵇ, sol_γˢʸᵐᵇ, data_γˢʸᵐᵇ = tssos_first(obj_γˢʸᵐᵇ, variables(obj_γˢʸᵐᵇ), QUIET=true, solution=true, newton=false);|

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.003173001708536059
Global optimality certified with relative optimality gap 0.000003%!


| (generic function with 36 methods)

In [21]:
solution = variables(obj_γˢʸᵐᵇ) => sol_γˢʸᵐᵇ
subs(Hˢʸᵐᵇ, solution)

2×2 Matrix{Term{true, Float64}}:
 25.2084  0.0
 0.0      0.0

In [22]:
subs(γ, solution)

2.4399885826169965